In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install --upgrade pip setuptools wheel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [ ]:
!pip install --upgrade scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade tensorflow-federated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.5/558.5 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
pip install tensorflow_federated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --quiet --upgrade tensorflow_federated
# !pip install --quiet tensorflow_federated==0.13.1

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import collections
import time
import pickle

import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
ROOT_DIR = '/content/gdrive/My Drive/Colab Notebooks'
data_dir = os.path.join(ROOT_DIR, 'preprocessed_data')

CLUSTER_NO = 2      # for cluster 2, change 'num_clients_per_round' to 2, for others change to 3.

output_dir = os.path.join(ROOT_DIR, 'output')
os.makedirs(output_dir, exist_ok=True)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
current_output_dir = os.path.join(output_dir, current_time+'_cluster_{}'.format(CLUSTER_NO))
os.makedirs(current_output_dir, exist_ok=True)

log_dir = os.path.join(current_output_dir, 'logs')
os.makedirs(log_dir, exist_ok=True)

In [ ]:
residential_cluster_dict = {-1:[7, 11, 15, 18, 20],
                            0:[3, 5, 14], 
                            1:[4, 8, 9, 10, 13, 19], 
                            2:[6, 12]}

# cluster_date_range_dict = {0: ['2015-02-21','2017-02-21'],
#                            1: [],
#                            2: []}

In [ ]:
residential_ids = residential_cluster_dict[CLUSTER_NO]
residential_ids

[6, 12]

In [ ]:
# Fixed here.
residential_ids = [7, 18, 5, 10, 13, 6]
valid_residential_ids = [11, 3, 4, 13, 19, 12]
# valid_residential_ids = [7, 18, 5, 10, 13, 6]

In [ ]:
time_range = ['2015-09-29','2017-09-29']

### Prepare data

In [ ]:
residential_df_list = []

for residential_id in residential_ids:

    processed_data_file = os.path.join(data_dir, 'preprocessed_station_{}.csv'.format(residential_id))
    residential_df = pd.read_csv(processed_data_file, 
                          parse_dates=['datetime'], 
                          infer_datetime_format=True, 
                          index_col='datetime', 
                          low_memory=False)
    
    residential_df_list.append(residential_df)

In [ ]:
valid_residential_df_list = []

for residential_id in valid_residential_ids:

    processed_data_file = os.path.join(data_dir, 'preprocessed_station_{}.csv'.format(residential_id))
    residential_df = pd.read_csv(processed_data_file, 
                          parse_dates=['datetime'], 
                          infer_datetime_format=True, 
                          index_col='datetime', 
                          low_memory=False)
    
    valid_residential_df_list.append(residential_df)

In [ ]:
residential_df_dict = {}
for i, residential_id in enumerate(residential_ids):
    residential_df_dict[residential_id] = residential_df_list[i]

for i, residential_id in enumerate(valid_residential_ids):
    residential_df_dict[residential_id] = valid_residential_df_list[i]

In [ ]:
#  residential_df_dict[residential_ids[0]].head()

In [ ]:
for residential_id in residential_ids:
    residential_df_dict[residential_id] = residential_df_dict[residential_id].resample('D').sum()

In [ ]:
# residential_df_dict[residential_ids[0]].head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)) # normalize features (0 ~ 1)

In [ ]:
features = ['energy_kWh']
targets = ['energy_kWh']

In [ ]:
for residential_id in residential_ids:
    residential_df = residential_df_dict[residential_id]
    residential_df[features] = scaler.fit_transform(residential_df[features])

In [ ]:
# residential_df_dict[residential_ids[0]].head()

In [ ]:
#time_range = cluster_date_range_dict[CLUSTER_NO]

start_date = time_range[0]
end_date =  time_range[1]

train_data_dict = {}
valid_data_dict = {}

for residential_id in residential_ids:
    residential_df = residential_df_dict[residential_id]
    train_data_dict[residential_id] = residential_df.loc[start_date:end_date]
    # valid_data_dict[residential_id] = residential_df.loc[end_date:]

In [ ]:
# Fixed here.

for residential_id in valid_residential_ids:
    residential_df = residential_df_dict[residential_id]
    valid_data_dict[residential_id] = residential_df.loc[end_date:]

In [ ]:
# for residential_id in residential_ids:
#     print(len(train_data_dict[residential_id]))
#     print(train_data_dict[residential_id].head())
#     train_data_dict[residential_id].plot()
#     print()

### Prepare data for tff


In [ ]:
window_size = 30
ahead = 1

In [ ]:
def window_dataset_for_tff(series, window_size, ahead=1, num_epochs=5, batch_size=256, shuffle_buffer=1000):

    def batch_format_fn(element):
        return collections.OrderedDict(
            x=tf.reshape(element[:-ahead], [window_size, len(features)]),
            y=tf.reshape(element[-ahead][0:len(targets)], [len(targets)]))

    # series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series.values)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.shuffle(shuffle_buffer)

    # dataset = dataset.map(lambda window: (window[:-ahead], window[-ahead]))
    dataset = dataset.map(batch_format_fn)

    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
tff_train_data_dict = {}
tff_valid_data_dict = {}

# Fixed here.
for residential_id in residential_ids:

    train_data = train_data_dict[residential_id]
    # valid_data = valid_data_dict[residential_id] 

    tff_train_data_dict[residential_id] = window_dataset_for_tff(train_data, window_size, ahead)
    # tff_valid_data_dict[residential_id] = window_dataset_for_tff(valid_data, window_size, ahead)           

In [ ]:
# Fixed here.
for residential_id in valid_residential_ids:
    valid_data = valid_data_dict[residential_id]
    tff_valid_data_dict[residential_id] = window_dataset_for_tff(valid_data, window_size, ahead)  

In [ ]:
tff_valid_data = tff_valid_data_dict[valid_residential_ids[0]]
# dummy_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(tff_valid_data)))

### Test federated model

In [ ]:
# Choose hyperpameters
lstm_layers = [256, 128] 
dense_layers = [64, 32]
dropout = 0.1
recurrent_dropout = 0.1
merge_mode = 'concat'

# Statistics collector
log_train_mse = []
log_test_mse = []
log_train_mae = []
log_test_mae = []

# Input and output shapes of model
input_shape = [window_size, len(features)]
output_shape = len(targets)

In [ ]:
def create_bi_LSTM_model(input_shape, output_shape, lstm_layers, dropout, recurrent_dropout, merge_mode='concat'):

    model = Sequential()

    for i in range(len(lstm_layers)):
        if (i+1) == (len(lstm_layers)):
            model.add(Bidirectional(LSTM(lstm_layers[i], activation='tanh', dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True), merge_mode=merge_mode))
        else:
            model.add(Bidirectional(LSTM(lstm_layers[i], activation='tanh', dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True), input_shape=input_shape))    
   
    model.add(Flatten())
    
    for j in range(len(dense_layers)):
        model.add(Dense(dense_layers[j]))
        model.add(Dropout(dropout))
        
    model.add(Dense(output_shape))
    
    return model

loss = tf.keras.losses.Huber()                          # Cannot input this directly into "tff.learning.from_keras_model()"
metrics = [tf.keras.metrics.MeanAbsoluteError()]        # RuntimeError: Attempting to capture an EagerTensor without building a function.

def model_fn():
    
    model = create_bi_LSTM_model(input_shape, output_shape, lstm_layers, dropout, recurrent_dropout, merge_mode)
    
    return tff.learning.from_keras_model(
        model,
        # dummy_batch=dummy_batch,
        input_spec= tff_valid_data.element_spec,
        loss=tf.keras.losses.Huber(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
client_optimizer_func = tf.keras.optimizers.Adam()        # Cannot input this directly into "tff.learning.from_keras_model()"
server_optimizer_func = tf.keras.optimizers.Adam()        # ValueError: Tensor("SGD/learning_rate:0", shape=(), dtype=resource) must be from the same graph as Tensor("zeros_like:0", shape=(1, 1024), dtype=float32).

federated_averaging = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda:  tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda:  tf.keras.optimizers.Adam())

AttributeError: ignored

In [ ]:
# Load tensorboard
%load_ext tensorboard
%tensorboard --logdir '{log_dir}'

In [ ]:
summary_writer = tf.summary.create_file_writer(log_dir)

In [ ]:
federated_train_data = []
federated_valid_data = []

for residential_id in residential_ids:

    tff_train_data = tff_train_data_dict[residential_id]
    # tff_valid_data = tff_valid_data_dict[residential_id] 

    federated_train_data.append(tff_train_data)
    # federated_valid_data.append(tff_valid_data)

In [ ]:
# Fixed here.
for residential_id in valid_residential_ids:
    tff_valid_data = tff_valid_data_dict[residential_id]
    federated_valid_data.append(tff_valid_data)

In [ ]:

# def select_clients(data_length, current_index, client_size=3):

#   selected_clients = []
#   for i in range(client_size):
#     if current_index < data_length:
#       selected_clients.append(current_index)
#     else:
#       current_index = 0
#       selected_clients.append(current_index)

#     current_index += 1
#   return selected_clients, current_index

# current_index = 0

# for i in range(20):
#   selected_clients, current_index = select_clients(len(federated_train_data), current_index, 3)
#   print(selected_clients)
#   selected_train_data = [federated_train_data[i] for i in selected_clients]
#   print(selected_train_data)

In [ ]:
def select_clients(data_length, current_index, client_size=3):

  selected_clients = []

  if current_index >= data_length:
    current_index = 0
    
  selected_clients.append(current_index)

  i = 1
  buf = current_index
  while len(selected_clients) < client_size:
    
    if (buf+i) < data_length:
      selected_clients.append(buf+i)  
    else:
      buf = 0
      i = 0
      selected_clients.append(buf)

    i+=1

  return selected_clients, current_index+1

# current_index = 0
# num_clients_per_round = 2
# for i in range(20):
#   selected_index, current_index = select_clients(len(federated_train_data), current_index, num_clients_per_round)
#   print(selected_index)

In [ ]:
federated_evaluation = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
state = federated_averaging.initialize()

In [ ]:
START_ROUND = 0
NUM_ROUNDS = 120

# cluster_0 = 40
# cluster_1 = 120
# cluster_2 = 40

In [ ]:
start_time = time.time()

current_index = 0
num_clients_per_round = 3
with summary_writer.as_default():
    for round_num in range(START_ROUND,(START_ROUND+NUM_ROUNDS)):

        selected_index, current_index = select_clients(len(federated_train_data), current_index, num_clients_per_round)
        selected_train_data = [federated_train_data[i] for i in selected_index]
        selected_valid_data = [federated_valid_data[i] for i in selected_index]

        print('round  {:2d}'.format(round_num))
        print('selected_index = {}'.format(selected_index))
        print('selected_clients = {}'.format([residential_ids[i] for i in selected_index]))

        state, train_metrics = federated_averaging.next(state, selected_train_data)
        print('train_metrics = {}'.format(train_metrics))
        log_train_mse.append(train_metrics['train']['loss'])
        log_train_mae.append(train_metrics['train']['mean_absolute_error'])

        # for name, value in train_metrics._asdict().items():
        #     tf.summary.scalar('train_' + str(name), value, step=round_num)

        for name, value in train_metrics['train'].items():
            tf.summary.scalar('train_' + str(name), value, step=round_num)

        test_metrics = federated_evaluation(state.model, selected_valid_data)
        print('test_metrics  = {}'.format(test_metrics))
        log_test_mse.append(test_metrics['loss'])
        log_test_mae.append(test_metrics['mean_absolute_error'])

        # for name, value in test_metrics._asdict().items():
        #     tf.summary.scalar('test_' + str(name), value, step=round_num)

        for name, value in test_metrics.items():
            tf.summary.scalar('test_' + str(name), value, step=round_num)

end_time = time.time()
elapsed_time = end_time - start_time

In [ ]:
training_history_file = os.path.join(current_output_dir, 'training_history.txt')

with open(training_history_file, 'wb') as file_pi:

        training_history = [np.array(log_train_mse).flatten(),
                            np.array(log_test_mse).flatten(),
                            np.array(log_train_mae).flatten(),
                            np.array(log_test_mae).flatten()]
        pickle.dump(training_history, file_pi)

training_history = pickle.load(open(training_history_file, "rb"))
train_mse, test_mse, train_mae, test_mae = training_history

### Evaluation


In [ ]:
federated_train_metrics = federated_evaluation(state.model, federated_train_data)
str(federated_train_metrics)

In [ ]:
federated_train_metrics['mean_absolute_error']

In [ ]:
federated_valid_metrics = federated_evaluation(state.model, federated_valid_data)
str(federated_valid_metrics)

In [ ]:
federated_valid_metrics['mean_absolute_error']

### Weight transfer

In [ ]:
model = create_bi_LSTM_model(input_shape, output_shape, lstm_layers, dropout, recurrent_dropout, merge_mode)
model.compile(loss=loss, 
              optimizer=client_optimizer_func,
              metrics=["mae"])

In [ ]:
tff.learning.assign_weights_to_keras_model(model, state.model)

In [ ]:
model_path = os.path.join(current_output_dir, 'federated_and_clustered_model_mae_{}.h5'.format(str('%.4f'%(federated_valid_metrics['mean_absolute_error']))))
tf.keras.models.save_model(model, model_path, save_format="h5")

### Prepare data for evaluation


In [ ]:
def window_dataset(series, window_size, ahead=1, batch_size=32, shuffle_buffer=1000): 

    # series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series.values)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-ahead], window[-ahead][0:len(targets)]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
def model_forecast(model, series, window_size):
    # series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series.values)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    dataset = dataset.batch(32).prefetch(1)
    forecast = model.predict(dataset)
    return forecast

In [ ]:
windowed_valid_set_dict = {}

for residential_id in residential_ids:

    valid_data = valid_data_dict[residential_id] 
    windowed_valid_set_dict[residential_id] =  window_dataset(valid_data, window_size, ahead)  

In [ ]:
valid_set = windowed_valid_set_dict[residential_ids[0]]
valid_metrics = model.evaluate(valid_set)

In [ ]:
valid_data = valid_data_dict[residential_ids[0]]
forecast = model_forecast(model, valid_data[:-1], window_size)[:,0]

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(valid_data.index[window_size:], valid_data[targets][window_size:], label='True', marker='o')
plt.plot(valid_data.index[window_size:], forecast, label='Prediction', marker='*',linestyle = ':')
plt.xlabel('Time-Slots')
plt.ylabel('Normalized Values')
plt.grid(True)
plt.legend()
img_path = os.path.join(current_output_dir, 'true_and_prediction_cluster_{}.png'.format(CLUSTER_NO))
plt.savefig(img_path, dpi=300, bbox_inches = 'tight')
plt.show()

In [ ]:
info_file = os.path.join(current_output_dir, 'info.txt')


info = '''
paradigm : federated and clustered (Contradictd)
cluster no : {}
activation='tanh' 
loss={}
client_optimizer={} 
server_optimizer={} 
metrics={}
federated train metrcis : {}
federated valid metrcis : {} 
valid metrcis : {} 
rounds : {}
num_of_clients_per_round : {} 
window : {} 
ahead : {} 
start_time : {}
end_time : {}
elapsed_time : {}
notes : residential ids = {}
      : time_range = {}
'''.format(CLUSTER_NO,
           loss,
           client_optimizer_func, 
           server_optimizer_func, 
           metrics,
           federated_train_metrics,
           federated_valid_metrics, 
           valid_metrics, 
           NUM_ROUNDS,
           num_clients_per_round,
           window_size, 
           ahead,
           start_time,
           end_time,
           elapsed_time,
           residential_ids,
           time_range)

with open(info_file, 'w') as f:
    f.write(info)